In [1]:
import transformers
import torch

In [2]:
from huggingface_hub import login
login()

In [3]:
user_prompt = """Collect information for a query plan using interleaving 'Thought', 'Action', and 'Observation' steps. Ensure you gather valid information related to transportation, dining, attractions, and accommodation. All information should be written in Notebook, which will then be input into the Planner tool. Note that the nested use of tools is prohibited. 'Thought' can reason about the current situation, and 'Action' can have 3 different types:

(1) AccommodationSearch[Budget,Preference]:
Description: Find the accommodation that matches the preference.
Parameters:
Preference: A list of preferences mentioned in the query.
Example: AccommodationSearch[Moderate Budget,[Good Location, Good Service]] would return the moderate price hotel that has a good or excellent location, as well as a good or excellent service.

(2) AttractionSearch[Budget, Preference]:
Description: Find the attractions that matches the preference.
Parameters:
Budget: The budget mentioned in the query.
Preference: A list of preferences mentioned in the query.
Example: AttractionSearch[Cheap budget,[Nature Oriented]] would return the cheap price and nature - oriented attractions.

(3) RestaurantSearch[Budget, Cuisine, Preference]:
Description: Find the restaurants that matches the preference.
Parameters:
Budget: The budget mentioned in the query.
Cuisine: The cuisine mentioned in the query.
Preference: A list of preferences mentioned in the query.
Example: RestaurantSearch[Expensive budget, Vietnamese, [Good Flavor, Good Value]] would return the expensive restaurants that offer Vietnamese cuisine, with good or excellent flavor and good or excellent value.

(4) Planner[Query]
Description: A smart planning tool that crafts detailed plans based on user input and the information stroed in Notebook.
Parameters: 
Query: The query from user.
Example: Planner[Give me a 3-day trip plan in Philadelphia] would return a detailed 3-day trip plan.

You should use as many as possible steps to collect engough information to input to the Planner tool. 

Each action only calls one function once. Do not add any description in the action.

Query: Please generate a detailed travel itinerary for a 3-day trip with a cheap budget. The plan should focus on activity-oriented attractions, include recommendations for Indian restaurants that offer good flavor and good value, and suggest accommodations that are in a good location. All relevant business information will be provided following. Ensure that all recommendations and their addresses are organized in chronological order for each day. Be concise and well-structured.
----- Example Starts -----
Day X:
- Accommodation: 
  - Name: XXXX
    Address: XXXX, XXXX

- Breakfast: 
  - Name: XXXX 
    Address: XXXX, XXXX

- Morning Attraction: 
  - Name: XXXX 
    Address: XXXX, XXXX 

- Lunch: 
  - Name: XXXX 
    Address: XXXX, XXXX 

- Afternoon Attraction: 
  - Name: XXXX 
    Address: XXXX, XXXX 
  - Name: XXXX 
    Address: XXXX, XXXX 

- Dinner: 
  - Name: XXXX 
    Address: XXXX, XXXX 

- Night Attraction: 
  - Name: XXXX 
    Address: XXXX, XXXX 
----- Example Ends ----- """

In [4]:
messages=[
    {"role": "user","content": user_prompt}
]

In [6]:
model_id = "meta-llama/Meta-Llama-3.1-70B-Instruct"
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cuda",
)


outputs = pipeline(
    messages,
    max_new_tokens=5000, # the llama 3.1 won't increase the detail with more token limit, neither decrese detail with less token, so just create a token limit that won't cut off the text.
    do_sample = False
)

print(outputs[0]["generated_text"][-1]['content'])

model-00030-of-00030.safetensors:  18%|#8        | 388M/2.10G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cuda


OutOfMemoryError: CUDA out of memory. Tried to allocate 128.00 MiB. GPU 0 has a total capacity of 39.38 GiB of which 85.81 MiB is free. Including non-PyTorch memory, this process has 39.18 GiB memory in use. Of the allocated memory 38.77 GiB is allocated by PyTorch, and 1.27 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)